<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d446c91890ba3521488132c3ce245359e66c55fb086ce4de33210ca628c97df9
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 14:42:05
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.38 C
-------------------
Today PnL: -1.97 L (-1.4%)
Current PnL: -23.15 L (-15.42%)
CY Booked + Current PnL: -11.25 L (-7.5%)
-------------------
Total profit:  1.14 L
Total loss:  -24.29 L
-------------------
Total Booked + Current PnL: 15.73 L (12.77%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.6%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 90.36 L (65.32%)
Deployed:  1.23 C
Current:  1.38 C
CAGR/XIRR %: 6.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.47,54.0,X-LC,3.73,211925.0,7425.0,15873.0,-1.07,3.63,7.49,11.39,10.0,0.47,1.54,12.91,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,96.80,50.0,M-SC,1.29,84790.0,-15987.0,16076.0,1.55,-15.86,18.96,0.09,245.0,-0.99,0.62,11.40,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.52,54.0,X-MC,1.04,208860.0,17118.0,20635.0,1.01,8.93,9.88,19.69,99.0,0.83,1.52,16.33,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,47.0,X-LC,2.85,214562.0,4196.0,24589.0,0.03,1.99,11.46,13.68,37.0,0.17,1.56,18.85,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,39.0,X-LC,2.75,194426.0,-5712.0,26597.0,-0.41,-2.85,13.68,10.44,4.0,-0.21,1.42,2.55,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.0,-36.43,64.0,M-SC,12.86,79144.0,-22015.0,96904.0,3.79,-21.76,122.44,74.03,202.0,-0.23,0.58,11.60,AR,BTT,AUTO
4,ANGELONE,3033.0,10.71,63.0,X-SC,10.31,195980.0,4974.0,52719.0,2.11,2.60,26.90,30.21,157.0,0.09,1.43,24.03,X40N,NTT,FINANCE
85,WIPRO,420.0,-11.98,55.0,M-LC,5.93,154411.0,3466.0,105988.0,1.60,2.30,68.64,72.51,53.0,0.03,1.13,8.46,XR,NTT,IT
76,TTKPRESTIG,770.0,96.80,50.0,M-SC,1.29,84790.0,-15987.0,16076.0,1.55,-15.86,18.96,0.09,245.0,-0.99,0.62,11.40,OX40N,NTT,DURABLES
17,CERA,9475.0,-23.15,44.0,H-SC,1.96,140277.0,-35626.0,77643.0,1.45,-20.25,55.35,23.89,149.0,-0.46,1.02,21.15,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-54.49,12.0,X-LC,1.07,144637.0,-131886.0,241963.0,-39.70,-47.69,167.29,39.81,54.0,-0.55,1.05,0.00,XY24,NTT,AUTO
81,VALIANTORG,838.0,-278.42,23.0,H-SC,3.10,99178.0,-64427.0,185741.0,-4.52,-39.38,187.28,74.15,139.0,-0.35,0.72,27.13,XR,NTT,CHEMICALS
23,DEN,75.0,89.12,37.0,M-SC,3.88,52896.0,-31568.0,67104.0,-3.95,-37.37,126.86,42.07,237.0,-0.47,0.39,12.33,AR,NTT,ENTERTAINMENT
10,BANDHANBNK,400.0,-12.63,48.0,H-SC,2.96,218018.0,-60541.0,317979.0,-3.43,-21.73,145.85,92.42,151.0,-0.19,1.59,26.00,XY24,NTT,BANKS
68,SURYODAY,240.0,57.89,54.0,H-SC,10.47,145636.0,-33435.0,99164.0,-3.12,-18.67,68.09,36.71,135.0,-0.34,1.06,44.50,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.49,147906.0,-1314.0,145495.0,1.19,-0.88,98.37,96.62,208.0,-0.01,1.08,60.07,XY25,NTT,FINANCE
59,SAIL,228.00,41.14,45.0,M-MC,12.50,222810.0,-2152.0,169804.0,-2.35,-0.96,76.21,74.53,192.0,-0.01,1.62,30.99,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.29,123410.0,74.0,131555.0,-2.92,0.06,106.60,106.72,119.0,0.00,0.90,23.45,AR,ATH,MISC
54,RECLTD,446.00,45.36,44.0,M-LC,6.19,204188.0,1018.0,41103.0,0.04,0.50,20.13,20.74,55.0,0.02,1.49,6.07,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,104.20,57.0,H-SC,11.55,129249.0,-13230.0,32752.0,-1.78,-9.29,25.34,13.70,163.0,-0.40,0.94,52.55,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,96.80,50.0,M-SC,1.29,84790.0,-15987.0,16076.0,1.55,-15.86,18.96,0.09,245.0,-0.99,0.62,11.40,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,47.0,H-SC,2.05,220509.0,-49158.0,85491.0,-0.28,-18.23,38.77,13.47,138.0,-0.58,1.61,12.12,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,44.0,H-SC,1.96,140277.0,-35626.0,77643.0,1.45,-20.25,55.35,23.89,149.0,-0.46,1.02,21.15,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-25.05,42.0,H-MC,7.02,101441.0,-30394.0,72622.0,-0.24,-23.05,71.59,32.03,98.0,-0.42,0.74,13.62,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,44.0,M-LC,6.19,204188.0,1018.0,41103.0,0.04,0.50,20.13,20.74,55.0,0.02,1.49,6.07,XY25,NTT,FINANCE
31,HINDZINC,730.22,35.55,68.0,M-LC,8.58,220654.0,15578.0,96999.0,0.34,7.60,43.96,54.89,52.0,0.16,1.61,33.52,X5K,ATH,METALS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.93,154411.0,3466.0,105988.0,1.60,2.30,68.64,72.51,53.0,0.03,1.13,8.46,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,52.0,M-LC,9.52,156338.0,15498.0,29001.0,-1.72,11.00,18.55,31.59,66.0,0.53,1.14,40.41,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,58.0,H-MC,12.66,191158.0,11350.0,101065.0,-2.03,6.31,52.87,62.52,88.0,0.11,1.39,42.55,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,35.55,68.0,M-LC,8.58,220654.0,15578.0,96999.0,0.34,7.60,43.96,54.89,52.0,0.16,1.61,33.52,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,58.0,H-MC,12.66,191158.0,11350.0,101065.0,-2.03,6.31,52.87,62.52,88.0,0.11,1.39,42.55,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.93,154411.0,3466.0,105988.0,1.60,2.30,68.64,72.51,53.0,0.03,1.13,8.46,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.29,123410.0,74.0,131555.0,-2.92,0.06,106.60,106.72,119.0,0.00,0.90,23.45,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-49.00,29.0,M-SC,3.42,86152.0,-5346.0,84093.0,-1.33,-5.84,97.61,86.07,223.0,-0.06,0.63,25.11,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,1.07,144637.0,-131886.0,241963.0,-39.70,-47.69,167.29,39.81,54.0,-0.55,1.05,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.38,29.0,X-MC,5.52,176005.0,-24501.0,121655.0,-1.29,-12.22,69.12,48.45,90.0,-0.20,1.28,1.49,X40N,ATH,FINANCE
28,HAVELLS,2069.16,-8.73,29.0,X-MC,1.37,226278.0,-21453.0,96508.0,-1.53,-8.66,42.65,30.30,92.0,-0.22,1.65,3.48,X40,ATH,ELECTRICAL
51,QUESS,986.00,-48.21,30.0,X-SC,37.85,51388.0,-13618.0,164544.0,-1.61,-20.95,320.20,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
5,ASIANPAINT,3465.66,-18.81,32.0,X-LC,5.32,204794.0,-46974.0,100185.0,-0.65,-18.66,48.92,21.13,27.0,-0.47,1.49,10.16,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.25,44.0,H-LC,0.95,155505.0,-30590.0,78064.0,1.06,-16.44,50.20,25.51,15.0,-0.39,1.13,14.60,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.52,54.0,X-MC,1.04,208860.0,17118.0,20635.0,1.01,8.93,9.88,19.69,99.0,0.83,1.52,16.33,XY25,NTT,AC
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,1.07,144637.0,-131886.0,241963.0,-39.70,-47.69,167.29,39.81,54.0,-0.55,1.05,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.73,29.0,X-MC,1.37,226278.0,-21453.0,96508.0,-1.53,-8.66,42.65,30.30,92.0,-0.22,1.65,3.48,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.58,41.0,X-MC,1.47,136693.0,560.0,30565.0,0.75,0.41,22.36,22.86,91.0,0.02,1.00,13.08,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,37.85,51388.0,-13618.0,164544.0,-1.61,-20.95,320.20,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,12.0,X-LC,1.07,144637.0,-131886.0,241963.0,-39.70,-47.69,167.29,39.81,54.0,-0.55,1.05,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.38,29.0,X-MC,5.52,176005.0,-24501.0,121655.0,-1.29,-12.22,69.12,48.45,90.0,-0.20,1.28,1.49,X40N,ATH,FINANCE
82,VBL,671.64,-21.70,38.0,X-LC,10.50,282156.0,-33686.0,146354.0,-0.05,-10.67,51.87,35.67,5.0,-0.23,2.06,1.71,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.64,36.0,X-MC,7.68,220280.0,-43085.0,156443.0,-0.83,-16.36,71.02,43.04,84.0,-0.28,1.61,2.27,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,37.85,51388.0,-13618.0,164544.0,-1.61,-20.95,320.20,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-40.94,35.0,X-SC,3.32,80473.0,-64687.0,140610.0,-0.90,-44.56,174.73,52.31,136.0,-0.46,0.59,7.72,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.58,34.0,X-SC,4.36,93267.0,-35403.0,80704.0,-2.20,-27.51,86.53,35.20,219.0,-0.44,0.68,7.05,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.49,44.0,X-SC,1.04,106190.0,4186.0,39736.0,-1.96,4.10,37.42,43.06,122.0,0.11,0.77,21.92,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.23,108195.0,-19743.0,66875.0,-0.25,-15.43,61.81,36.84,143.0,-0.30,0.79,10.77,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.97,43.0,X-LC,12.29,278710.0,-67246.0,135481.0,-1.40,-19.44,48.61,19.73,1.0,-0.50,2.03,2.65,X40,ATH,IT
39,INFY,2275.00,-18.52,52.0,X-LC,7.63,318052.0,5164.0,166532.0,-1.43,1.65,52.36,54.87,3.0,0.03,2.32,8.41,X40,BTT,IT
41,ITC,452.00,-39.61,39.0,X-LC,2.75,194426.0,-5712.0,26597.0,-0.41,-2.85,13.68,10.44,4.0,-0.21,1.42,2.55,X40,NTT,FMCG
82,VBL,671.64,-21.70,38.0,X-LC,10.50,282156.0,-33686.0,146354.0,-0.05,-10.67,51.87,35.67,5.0,-0.23,2.06,1.71,X40N,ATH,FMCG
1,ABB,7934.00,-41.13,41.0,H-LC,7.28,245832.0,-15787.0,135011.0,-0.51,-6.03,54.92,45.57,7.0,-0.12,1.79,4.60,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.71,80640.0,-32909.0,72963.0,-1.39,-28.98,90.48,35.27,268.0,-0.45,0.59,97.37,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,38.0,H-SC,9.90,89220.0,-8760.0,30362.0,-1.47,-8.94,34.03,22.05,152.0,-0.29,0.65,29.69,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,104.20,57.0,H-SC,11.55,129249.0,-13230.0,32752.0,-1.78,-9.29,25.34,13.70,163.0,-0.40,0.94,52.55,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,10.47,145636.0,-33435.0,99164.0,-3.12,-18.67,68.09,36.71,135.0,-0.34,1.06,44.50,XR,NTT,BANKS
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.49,147906.0,-1314.0,145495.0,1.19,-0.88,98.37,96.62,208.0,-0.01,1.08,60.07,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.49,147906.0,-1314.0,145495.0,1.19,-0.88,98.37,96.62,208.0,-0.01,1.08,60.07,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,10.47,145636.0,-33435.0,99164.0,-3.12,-18.67,68.09,36.71,135.0,-0.34,1.06,44.50,XR,NTT,BANKS
31,HINDZINC,730.22,35.55,68.0,M-LC,8.58,220654.0,15578.0,96999.0,0.34,7.60,43.96,54.89,52.0,0.16,1.61,33.52,X5K,ATH,METALS
50,PGHH,17907.65,-26.31,61.0,X-MC,2.91,213300.0,12480.0,55309.0,0.29,6.21,25.93,33.76,80.0,0.23,1.55,11.73,X40,ATH,FMCG
63,SHALBY,327.00,1163.16,52.0,M-SC,22.89,171760.0,-9383.0,54843.0,-2.75,-5.18,31.93,25.10,235.0,-0.17,1.25,37.65,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,44.01
2,50,75.83


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.44
LC    33.70
MC    21.87
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.67
X40      14.85
XY25     12.13
XR       10.98
X40N     10.09
AR        8.23
OX40N     7.70
X200      1.80
X5K       1.61
SR        1.12
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.63
X-LC    22.58
X-MC    16.14
M-SC    12.74
M-LC     5.37
H-LC     4.72
X-SC     4.63
H-MC     3.80
M-MC     1.62
L-SC     1.44
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.08,-6.36,41.42
IT,13.06,-17.23,80.73
FINANCE,12.18,-13.17,64.47
MISC,6.91,-20.97,88.00
BANKS,6.43,-13.00,72.44
PAINTS,5.62,-19.91,37.79
ELECTRICAL,5.36,-11.90,51.16
INSURANCE,3.84,-5.05,44.32
AUTO,2.84,-47.63,118.21


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3364497.0,22
XR,1403669.0,14
AR,1199722.0,9
X40,803016.0,10
X40N,642227.0,8
OX40N,589435.0,10
XY25,519898.0,8
SR,267530.0,2
X5K,96999.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3482253.0,25
M-SC,1294668.0,15
X-LC,1141814.0,13
X-MC,1113377.0,11
X-SC,545188.0,6
H-MC,358427.0,3
H-LC,291147.0,3
M-LC,273091.0,4
L-SC,262959.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1214511.0      6
           XR         830081.0      7
           AR         822639.0      5
M-SC       XY24       701541.0      6
X-MC       XY24       589174.0      4
X-LC       X40        496842.0      5
           XY24       311182.0      2
X-SC       X40N       299940.0      4
H-SC       OX40N      276944.0      4
           SR         267530.0      2
X-MC       X40        225470.0      4
X-LC       X40N       220632.0      3
H-LC       AR         213075.0      2
H-MC       XY24       184740.0      1
X-MC       XY25       177078.0      2
L-SC       XR         172922.0      2
M-MC       XY24       169804.0      1
X-SC       XY24       164544.0      1
M-SC       AR         164008.0      2
           OX40N      149832.0      4
           XY25       145495.0      1
           XR         133792.0      2
X-MC       X40N       121655.0      1
X-LC       XY25       113158.0      3
M-LC       XR         105988.0      1
H-MC       XR         101065.0      1
M-LC       X5K         96999.0      1
L-SC       OX40N       90037.0      1
X-SC       X40         80704.0      1
H-LC       X200        78072.0      1
H-MC       OX40N       72622.0      1
H-SC       MH          70548.0      1
L-MC       XR          59821.0      1
L-LC       XY25        43064.0      1
M-LC       XY25        41103.0      1
           XY24        29001.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
